# GroupBy Ecommerce Analysis
Jake Dunn - Feb 22, 2020

## Load and Preprocess Data

In [12]:
import pandas as pd
import json
import sklearn
from scipy.stats import chi2, chi2_contingency, chisquare

In [13]:
data = pd.read_csv('./resources/data-1.csv')
data.head()

,customer_integrated_id,refinement,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate
0,cjsyuvlck00013ra31lxcso69,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",3,0,0,0,0.0,0.0
1,cjwti47ac0001458du5ow5ikh,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",3,0,0,0,0.0,0.0
2,013146471,"{""name"":""visualVariant.nonvisualVariant.gbi_Me...",5,0,0,0,0.0,0.0
3,014182827,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",2,0,0,0,0.0,0.0
4,023424641,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",2,0,0,0,0.0,0.0


In [14]:
print(data.shape)

(168362, 8)


In [15]:
refinement_df = pd.json_normalize(data['refinement'].apply(json.loads))
refinement_df.head()

,name,displayName,value,high,low
0,gbi_categories.1,Category,Vitamin C,NaN,NaN
1,visualVariant.nonvisualVariant.BRAND,Brands,Jason,NaN,NaN
2,visualVariant.nonvisualVariant.gbi_Merged_concern,Concern,Eye/Macular Health,NaN,NaN
3,visualVariant.nonvisualVariant.BRAND,Brands,Swanson Health Products,NaN,NaN
4,gbi_categories.1,Category,CoQ10,NaN,NaN


In [16]:
df = pd.concat([data, refinement_df], axis=1)
df.head(5)

,customer_integrated_id,refinement,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,name,displayName,value,high,low
0,cjsyuvlck00013ra31lxcso69,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",3,0,0,0,0.0,0.0,gbi_categories.1,Category,Vitamin C,NaN,NaN
1,cjwti47ac0001458du5ow5ikh,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",3,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Jason,NaN,NaN
2,013146471,"{""name"":""visualVariant.nonvisualVariant.gbi_Me...",5,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.gbi_Merged_concern,Concern,Eye/Macular Health,NaN,NaN
3,014182827,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",2,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Swanson Health Products,NaN,NaN
4,023424641,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",2,0,0,0,0.0,0.0,gbi_categories.1,Category,CoQ10,NaN,NaN


In [17]:
x_tab = pd.crosstab([df['displayName'], df['name']], columns='count')
x_tab

col_0                                                                                                  count
displayName                                        name                                                     
Brands                                             visualVariant.nonvisualVariant.BRAND                24149
Category                                           gbi_categories.1                                    80599
                                                   gbi_categories.2                                    40468
                                                   gbi_categories.3                                      831
Concern                                            visualVariant.nonvisualVariant.gbi_Merged_concern    4737
Flavor                                             visualVariant.nonvisualVariant.gbi_Merged_flavor      169
Form                                               visualVariant.nonvisualVariant.FORM                   616
                                                   visualVariant.nonvisualVariant.gbi_Merged_form       2603
More Ways to Browse                                visualVariant.nonvisualVariant.MOREWAYSTOBROWSE      5587
Pill Size                                          visualVariant.nonvisualVariant.PILLSZ                  34
Potency                                            visualVariant.nonvisualVariant.POTENT                2811
Specialty                                          visualVariant.nonvisualVariant.SDP                    564
                                                   visualVariant.nonvisualVariant.gbi_Merged_speci...   3666
Swanson Brands                                     visualVariant.nonvisualVariant.BRAND2                1474
visualVariant.nonvisualVariant.BAZARRATE           visualVariant.nonvisualVariant.BAZARRATE               25
visualVariant.nonvisualVariant.BAZARREVIEW         visualVariant.nonvisualVariant.BAZARREVIEW             24
visualVariant.nonvisualVariant.MOREWAYSTOBROWSE    visualVariant.nonvisualVariant.MOREWAYSTOBROWSE         4
visualVariant.nonvisualVariant.gbi_Merged_speci... visualVariant.nonvisualVariant.gbi_Merged_speci...      1

In [18]:

stat, p, dof, expected = chi2_contingency(x_tab)

In [19]:
print('dof=%d' % dof)
print(expected)

dof=0
[[2.4149e+04]
 [8.0599e+04]
 [4.0468e+04]
 [8.3100e+02]
 [4.7370e+03]
 [1.6900e+02]
 [6.1600e+02]
 [2.6030e+03]
 [5.5870e+03]
 [3.4000e+01]
 [2.8110e+03]
 [5.6400e+02]
 [3.6660e+03]
 [1.4740e+03]
 [2.5000e+01]
 [2.4000e+01]
 [4.0000e+00]
 [1.0000e+00]]


In [20]:
prob = .95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

probability=0.950, critical=nan, stat=0.000


In [21]:
chisquare(x_tab)

Power_divergenceResult(statistic=array([772723.86695335]), pvalue=array([0.]))

In [22]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
#vec.fit_transform().toarray()
#vec.get_feature_names()